# Prompt Engineering Example (using Mistral-7B)
#### Author: Arnav Ahuja

This notebook serves as a basic reference to help you get started with the **prompt engineering task** assigned under the LLM team. It is designed to demonstrate how to build and test different prompting strategies using an LLM.

## Scope

You are expected to work on the following **two use cases**:
- **AI Content Detection**
- **Feedback Generation**

This notebook provides basic code examples for testing prompt formats across:
- **Zero-Shot Prompting**
- **Few-Shot Prompting**
- **Chain-of-Thought (CoT) Reasoning**
- **Role-Based Prompting**
- **Structured Output Formatting**

> Note: This is a **light ChatGPT-style baseline** modified slightly to demonstrate intent. It is **not a complete implementation** and **should not be copied directly**. You are expected to use this structure to experiment with different models and build your own review pipeline.


## Requirements

- You must build a working inference pipeline using Python.
  - For **open models** (e.g., Mistral, TinyLlama, Qwen): use Hugging Face or similar frameworks
  - For **closed/commercial models** (e.g., Gemini, OpenAI): use official APIs with authentication

- You must test and compare **multiple prompt engineering techniques** for both use cases above.

- For each prompt tested, **benchmark performance** using your judgment:
  - Use an **Output Quality Rating (1–5)** to capture how well the model performed
    - 1 = irrelevant/incorrect, 3 = usable but limited, 5 = ideal
  - Note whether the model was **accurate**, **concise**, **verbose**, or **unclear**

- Explore:
  - Prompt structures (system vs user messages)
  - How the model reacts to task instructions
  - Whether certain prompt types (e.g., CoT) improve reliability or justification


## Deliverables

- A Python notebook or script with:
  - A functional model inference pipeline
  - At least **3 prompts per use case**, each showing a different prompting technique

- A short documentation file (`.md` or part of notebook) explaining:
  - How to run your code
  - How to obtain any required API keys or model access tokens
  - Observations about model behavior, quirks, and performance for the task
  

## Setup Notes

- Prompt length impacts cost and performance; be concise where possible.
- Deakin offers **Azure OpenAI** access (free GPT-3.5/4 quota). Explore this as an option.
- If you use Hugging Face models that require a token, make sure to document the model and authentication steps.

Let me know if you need a starter notebook, test prompts, or help selecting an open-source model.

### Loading the model using HuggingFace
#### Recommended to use GCP for faster download

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

# Set Hugging Face token (DO NOT expose this publicly in production)
HF_TOKEN = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxx"
os.environ["HF_TOKEN"] = HF_TOKEN

# Load Mistral model with authentication
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    token=HF_TOKEN
)


Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## 1. AI Detection

In [2]:
# Common input
submission_text = "I asked ChatGPT for ideas and then rewrote the draft in my own words with added examples."

# Few-shot examples
few_shot_examples = """
Submission: "I completed the assignment using my own understanding and research."
Analysis: Natural tone, subjective phrasing, and small grammar issues indicate human writing.
Label: Human

Submission: "This response was generated using ChatGPT based on the prompt."
Analysis: Perfect grammar, vague structure, and lack of subjectivity suggest AI-generated text.
Label: AI

Submission: "I used ChatGPT for brainstorming, then wrote the essay in my voice."
Analysis: Mixed signals of AI-style structure and personal voice suggest a Hybrid.
Label: Hybrid
"""

# Inference function
def generate(prompt, label=""):
    print(f"\n=== {label} PROMPT ===\n")
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to("cuda")
    output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    print(result.split("Submission:")[-1].strip())

# 1 Zero-Shot Prompt
zero_shot_prompt = f"""
You are an AI detector. Classify the following submission as Human, AI, or Hybrid.

Submission: "{submission_text}"
Label:"""

# 2 Few-Shot Prompt
few_shot_prompt = f"""
You are an AI classifier. Below are examples of submissions with analysis and labels.

{few_shot_examples}

Submission: "{submission_text}"
Analysis:"""

# 3 Role Prompting + Chain-of-Thought
cot_prompt = f"""
You are an AI text detection expert. Your task is to examine writing style, coherence, tone, and structure to classify the source.

Submission: "{submission_text}"
Think step by step before deciding the label.
"""

# 4 Structured Output Format
structured_prompt = f"""
You are a classifier for AI-generated vs. Human vs. Hybrid submissions.

Submission: "{submission_text}"
Return your response in JSON format:
{{ "label": "<Human/AI/Hybrid>", "justification": "<short explanation>" }}
"""

# 5 Combined Strategy: Role + CoT + Few-Shot + JSON Output
combined_prompt = f"""
You are an AI text detection expert. Analyze student writing to determine if it's Human, AI-generated, or Hybrid (AI-assisted).
Use reasoning and provide structured output.

Here are examples:
{few_shot_examples}

Now classify the following:

Submission: "{submission_text}"
Respond in JSON format as:
{{ "label": "<Human/AI/Hybrid>", "justification": "<short explanation>" }}
"""

# Run all types
generate(zero_shot_prompt, "ZERO-SHOT")
generate(few_shot_prompt, "FEW-SHOT")
generate(cot_prompt, "ROLE + COT")
generate(structured_prompt, "STRUCTURED")
generate(combined_prompt, "COMBINED")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== ZERO-SHOT PROMPT ===



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"I asked ChatGPT for ideas and then rewrote the draft in my own words with added examples."
Label: Hybrid. The submission indicates that the person used an AI (ChatGPT) as a source of inspiration or ideas, but they still wrote the final draft in their own words and added examples.

=== FEW-SHOT PROMPT ===



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"I used Chat

=== ROLE + COT PROMPT ===



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"I asked ChatGPT for ideas and then rewrote the draft in my own words with added examples."
Think step by step before deciding the label.

1. Writing Style: The writing style is clear, concise, and grammatically correct.
2. Coherence: The text is coherent and logical. The writer is explaining a process they went through to create content.
3. Tone: The tone is neutral and informative.
4. Structure: The text has a simple structure with a clear beginning, middle, and end.

Based on the analysis, the label for this text would be "Academic or Informative Writing". This label is suitable because the text is written in a formal and objective manner, and it provides information about a process.

=== STRUCTURED PROMPT ===



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"I asked ChatGPT for ideas and then rewrote the draft in my own words with added examples."
Return your response in JSON format:
{ "label": "<Human/AI/Hybrid>", "justification": "<short explanation>" }

{ "label": "Hybrid", "justification": "The submission indicates that the AI was used as a tool for generating ideas, but the final draft was written by the human and included their own words and examples." }

=== COMBINED PROMPT ===

"I asked ChatGPT for ideas and then rewrote the draft in my own words with added examples."
Respond in JSON format as:
{ "label": "<Human/AI/Hybrid>", "justification": "<short explanation>" }

{
 "label": "Hybrid",
 "justification": "The student acknowledges using ChatGPT for ideas, but also mentions rewriting the draft in their own words and adding examples, indicating a human touch."
}


## 2. Feedback Generation

In [3]:
# Common Input
assignment_prompt = "Discuss the causes of World War I in 500 words."
rubric = """
1. Thesis Clarity
2. Historical Accuracy
3. Argument Structure
4. Writing Mechanics
"""
student_submission = """
World War I started due to many reasons like nationalism and secret alliances. Serbia and Austria-Hungary had a fight, and because of that everyone started joining in. Also, countries wanted to be stronger than each other, so they started a war. It was also due to the assassination of Franz Ferdinand.
"""

# Shared evaluation function
def generate(prompt, label=""):
    print(f"\n=== {label} PROMPT ===\n")
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to("cuda")
    output = model.generate(**inputs, max_new_tokens=300, do_sample=False)
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    print(result.split("Student Submission:")[-1].strip())

# 1 Zero-Shot Prompt
zero_shot_prompt = f"""
You are a university writing instructor.

Assignment: {assignment_prompt}
Rubric:
{rubric}

Student Submission:
{student_submission}

Please provide constructive feedback based on the rubric.
"""

# 2 Few-Shot Prompt
few_shot_examples = """
Example:

Assignment: Discuss the impact of climate change on agriculture.
Rubric:
1. Clarity
2. Evidence
3. Structure
4. Grammar

Student Submission:
Climate change make farms worse because no rain and too much sun. Crops are dying everywhere. Farmers are struggling.

Feedback:
- Clarity: The message is understandable but could benefit from a clearer thesis statement.
- Evidence: Lacks specific data or examples to support the claims.
- Structure: Points are listed loosely; consider organizing paragraphs by theme.
- Grammar: Several grammatical errors and awkward phrasing.
"""

few_shot_prompt = f"""
You are a writing tutor providing feedback aligned to the rubric.

{few_shot_examples}

Now evaluate the following:

Assignment: {assignment_prompt}
Rubric:
{rubric}

Student Submission:
{student_submission}

Provide feedback per rubric.
"""

# 3 Role Prompt + Chain-of-Thought
cot_prompt = f"""
You are a university instructor. Analyze each rubric point step by step and provide constructive feedback.

Assignment: {assignment_prompt}
Rubric:
{rubric}

Student Submission:
{student_submission}

First list key strengths and weaknesses, then give full feedback.
"""

# 4 Structured Output
structured_prompt = f"""
You are a grading assistant. Evaluate the student work based on each rubric criterion.

Assignment: {assignment_prompt}
Rubric:
{rubric}

Student Submission:
{student_submission}

Respond in this format:
Thesis Clarity: ...
Historical Accuracy: ...
Argument Structure: ...
Writing Mechanics: ...
"""

# 5 Combined Strategy: Role + Few-shot + Structure
combined_prompt = f"""
You are a supportive academic writing tutor. Below is an example of how to provide rubric-aligned feedback.

{few_shot_examples}

Now assess the following:

Assignment: {assignment_prompt}
Rubric:
{rubric}

Student Submission:
{student_submission}

Provide rubric-aligned feedback in this format:
Thesis Clarity: ...
Historical Accuracy: ...
Argument Structure: ...
Writing Mechanics: ...
"""

# Run all strategies
generate(zero_shot_prompt, "ZERO-SHOT")
generate(few_shot_prompt, "FEW-SHOT")
generate(cot_prompt, "ROLE + COT")
generate(structured_prompt, "STRUCTURED OUTPUT")
generate(combined_prompt, "COMBINED STRATEGY")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== ZERO-SHOT PROMPT ===



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


World War I started due to many reasons like nationalism and secret alliances. Serbia and Austria-Hungary had a fight, and because of that everyone started joining in. Also, countries wanted to be stronger than each other, so they started a war. It was also due to the assassination of Franz Ferdinand.


Please provide constructive feedback based on the rubric.


Feedback:

1. Thesis Clarity: The student's thesis is clear, stating that World War I was caused by multiple factors including nationalism, secret alliances, the desire for power, and the assassination of Franz Ferdinand. However, the thesis could be more specific and detailed to better guide the essay.

2. Historical Accuracy: The student's information is generally correct, but the explanation of the causes is not comprehensive. The essay could benefit from more specific examples and details to support the claims.

3. Argument Structure: The student presents the causes of World War I in a logical order, but the essay could ben

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


World War I started due to many reasons like nationalism and secret alliances. Serbia and Austria-Hungary had a fight, and because of that everyone started joining in. Also, countries wanted to be stronger than each other, so they started a war. It was also due to the assassination of Franz Ferdinand.


Provide feedback per rubric.

Feedback:

- Thesis Clarity: The student has presented a clear thesis statement, acknowledging multiple causes of World War I.
- Historical Accuracy: The student's explanation is generally accurate, but it could benefit from more specific details and examples.
- Argument Structure: The student has presented the causes in a logical order, but the paragraph could be expanded to include more depth and analysis.
- Writing Mechanics: The student's writing is clear and easy to follow, but there are some grammatical errors and awkward phrasing that should be corrected. For example, "it was also due to" could be replaced with "additionally, it was caused by" or "an

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


World War I started due to many reasons like nationalism and secret alliances. Serbia and Austria-Hungary had a fight, and because of that everyone started joining in. Also, countries wanted to be stronger than each other, so they started a war. It was also due to the assassination of Franz Ferdinand.


First list key strengths and weaknesses, then give full feedback.

Strengths:
- The student acknowledges multiple causes of World War I, including nationalism, secret alliances, and the assassination of Franz Ferdinand.
- The student provides a brief summary of the conflict between Serbia and Austria-Hungary.

Weaknesses:
- The student's thesis statement is not clear. It is stated as "World War I started due to many reasons," but it is not explicitly stated what those reasons are.
- The student's argument lacks structure. The causes are mentioned in a random order, and there is no clear explanation of how they are related to each other or to the overall cause of the war.
- The student's

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


World War I started due to many reasons like nationalism and secret alliances. Serbia and Austria-Hungary had a fight, and because of that everyone started joining in. Also, countries wanted to be stronger than each other, so they started a war. It was also due to the assassination of Franz Ferdinand.


Respond in this format:
Thesis Clarity: ...
Historical Accuracy: ...
Argument Structure: ...
Writing Mechanics: ...

Thesis Clarity: The student's thesis is clear, as they state that World War I had multiple causes, including nationalism, secret alliances, and the assassination of Franz Ferdinand. However, the thesis could be more specific and detailed, outlining the exact role each cause played in the war's outbreak.

Historical Accuracy: The student correctly identifies nationalism and secret alliances as causes of World War I. They also correctly state that the assassination of Franz Ferdinand was a trigger event. However, the student's explanation of why countries wanted to be stron